Before you turn this assignment in, make sure everything runs as expected by going to the menubar and running: 

**Kernel $\rightarrow$ Restart & Run All**

Please replace all spots marked with `# ADD YOUR CODE HERE` or `ADD YOUR ANSWER HERE`.

And start by filling in your name and student_id below:

In [ ]:
NAME = ""
STUDENT_ID = ""

In [ ]:
assert len(NAME) > 0, "Please fill in your name"
assert len(STUDENT_ID) > 0, "Please fill in your student id"

---

In [ ]:
# If you encounter import issues with scipy,
# please make sure to update Gensim to 4.3.3
%pip install --upgrade --quiet gensim==4.3.3

In [ ]:
import doctest

import gensim.downloader
import numpy as np
import pandas as pd

from html import unescape
from typing import Callable, List, Dict , Tuple

from gensim.models.keyedvectors import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def test(fn: Callable):
    if __name__ == "__main__":
        doctest.run_docstring_examples(fn, globals(), verbose=True, name=fn.__name__, optionflags=doctest.ELLIPSIS)

# Week 5 - Word Embeddings & Recommender Systems

Welcome to the last assignment of Zoekmachines! 👋

In the first part of this assignment, we look into word embeddings covered in last week's lecture.

Part two and three cover recommender systems, the basis for systems that recommend our movies, music, recipes, and even news. So far in this course, we have looked into using the textual content of queries and documents to find relevant items. For example, recommending a news article because of its high tf-idf match with something you previously engaged with would be a content-based recommender system. Many of the techniques we have learned in this course are helpful for this approach. This week, however, we will look into finding similar items not by inspecting their textual content but rather from their user interaction patterns. This is called the collaborative filtering approach to recommendation.

As always, for any questions, problems, or feedback please contact your TA. Good luck with the assignment!


### Resources
- 📚 [Neighborhood-based Recommendation Methods - Desrosiers and Karypis, RecSys Handbook Chapter 4.2](https://www.cse.iitk.ac.in/users/nsrivast/HCC/Recommender_systems_handbook.pdf)

- 📚 [Gensim Word2Vec tutorial](https://rare-technologies.com/word2vec-tutorial/)

# Part I - Word Embeddings

In last week's lecture, you learned about semantic matching and word embeddings. Word embeddings took off in the NLP community in 2013 with the introduction of the Word2Vec model, which is a neural network trained to predict the missing word given its surrounding context words (or the other way around: given a word, predict the context words).

Just by executing this rather simple task on a large amount of documents, the model learned surprisingly powerful word representations. The author's discovered, that Word2Vec embeddings encoded:

* Syntactic similarities bewteen words, e.g.: `[go, went, gone]` or `[kind, kinder, kindest]`
* Semantic relationships, e.g.: `[brother, sister, grandson, granddaughter]`, [`germany`, `germans`, `netherlands`, `dutch`]
* And they encoded analogies in the latent space, e.g.: `woman` is to `man`, as `queen` is to `king`.

A word of caution: While these anaolgies computed using vector arithmetics (adding or subtracting word vectors), were quite impressive, they also often revealed and encoded stereotypes that were present in the training dataset.

With that said, let's explore word embeddings a bit. First, let's download a small word embedding corpus that was trained on news articles from the early 2000s. Throughout these tasks, we will use the [Gensim library](https://tedboy.github.io/nlps/generated/generated/gensim.models.Word2Vec.html) (which is also quite useful for using more advanced topic models than LSI), feel free to use all available API methods in the following tasks.

<div class="alert alert-warning">
💡 The word embeddings will be downloaded into ~/gensim-data in your home directory. Feel free to delete it right after this task.
</div>

In [ ]:
if __name__ == "__main__":
    model = gensim.downloader.load("glove-wiki-gigaword-100")

## 1.1 Nearest neighbors

📝 As your first task, find the **top-n most similar terms** to a **list of query terms** using [Gensim](https://tedboy.github.io/nlps/generated/generated/gensim.models.Word2Vec.html):

- Most similar here means words with the highest cosine similarity to the query vector.
- If multiple query terms are given, find words with the similarity to the **average word vector of all query terms**.
- Return the list of words in order of similarity but without the similarity scores themselves.
- The query terms themselves should not appear in the list of most similar terms.

### Example
The five closest terms to `["hangover"]` are `['fright', 'headache', 'letdown', 'mania', 'affliction']`,

and the closest terms to `["hangover", "movie"]` are `['hollywood', 'movies', 'blockbuster', 'sequel', 'film']`.

<div class="alert alert-warning">
💡 You can access a vector of a word using model["movie"]. Inspect the Gensim API to find ways to compute cosine similarity and perform nearest neighbor search.
</div>

In [ ]:
def find_most_similar(model: KeyedVectors, query_terms: List[str], top_k=5):
    """
    >>> find_most_similar(model, ["hangover"], top_k=3)
    ['fright', 'headache', 'letdown']
    
    >>> find_most_similar(model, ["hangover", "movie"], top_k=8)
    ['hollywood', 'movies', 'blockbuster', 'sequel', 'film', 'nightmare', 'comedy', 'horror']
    
    >>> find_most_similar(model, ["clinton"], top_k=3)
    ['bush', 'obama', 'gore']
    
    >>> find_most_similar(model, ["hillary", "clinton"], top_k=5)
    ['rodham', 'obama', 'barack', 'bush', 'mccain']
    """
    results = []
    
    # ADD YOUR CODE HERE
    
    return results

In [ ]:
test(find_most_similar)

## 1.2 Analogies

📝 In this second tasks, we will explore the word analogies encoded in our word vectors using vector arithmetics.

- Given an example pair of words (e.g., `["man", "woman"]`), find the anaolgy for a new term (e.g., `["king", "?"]`).
- Return the top answer for your analogy.

### Resources

📚 [Gensim Word2Vec tutorial](https://rare-technologies.com/word2vec-tutorial/)

In [ ]:
def find_analogy(model: KeyedVectors, word_pair: Tuple[str, str], term):
    """
    >>> find_analogy(model, word_pair=["man", "woman"], term="king")
    'queen'
    
    >>> find_analogy(model, word_pair=["democrat", "republican"], term="obama")
    'bush'
    
    >>> find_analogy(model, word_pair=["go", "going"], term="walk")
    'walking'
    
    >>> find_analogy(model, word_pair=["europe", "euro"], term="america")
    'dollar'
    
    >>> find_analogy(model, word_pair=["germany", "berlin"], term="netherlands")
    'amsterdam'
    """
    result = ""
    
    # ADD YOUR CODE HERE
    
    return result

In [ ]:
test(find_analogy)

# Part II - User-User Collaborative Filtering

Next, we will move to recommender systems for part II and III of this assignment, where we are building a recommender system for books. First, we look into a user-user collaborative filtering approach, the idea that users might like content that users with a similar preference previously already enjoyed. Specifically, we look into nearest neighbor collaborative filtering, a very basic approach to solve this problem. Essentially: Given a user, find users that are very similar to them (called neighbors) and recommend content that these other users enjoyed but our user has not engaged with.

For this assignment, we use a public book dataset with ≈94k real user ratings for ≈13k books on a scale from 1 - 11 (you can treat zeros as missing values throughout this exercise). Let's begin by downloading the dataset:

In [ ]:
def load_data(url: str = "https://raw.githubusercontent.com/irlabamsterdam/uva-ir0-assignments/main/data/rating.csv") -> pd.DataFrame:
    return pd.read_csv(url)

def get_id2title(df: pd.DataFrame) -> Dict[int, str]:
    return df.set_index("item_id").title.to_dict()

if __name__ == "__main__":
    df = load_data()
    id2title = get_id2title(df)
    print(df.head())


## 2.1 The user-item-rating matrix

We begin with a classic way to represent rating data, as a 2D matrix between users and items.

📝 Transform our dataset into a rating matrix (of shape users x items), containing a row for each user and a column for each item. Insert the ratings as entries in this 2D array. If a user has not rated a book, the entry should be zero.

<div class="alert alert-warning">
💡 Pandas's "pivot_table" operation might come in handy to speed up this transformation.
</div>

In [ ]:
def get_user_item_ratings(df: pd.DataFrame) -> np.ndarray:
    """
    >>> user_item_ratings = get_user_item_ratings(df)
    >>> user_item_ratings.shape
    (1710, 13053)
    >>> user_item_ratings[0, [320, 444, 1288, 1406, 1774]]
    array([11.,  8.,  1.,  1.,  1.])
    >>> user_item_ratings[1709, [10703, 10823, 11208, 12678, 12937]]
    array([9., 8., 1., 1., 9.])
    >>> user_item_ratings[[0, 1, 2, 3, 4], [1881, 9433, 18, 9964, 888]]
    array([ 8.,  7., 10.,  1., 10.])
    """
    user_item_ratings = np.array([[]])
    
    # ADD YOUR CODE HERE
    return user_item_ratings.astype(float)

In [ ]:
if __name__ == "__main__":
    test(get_user_item_ratings)
    user_item_ratings = get_user_item_ratings(df)

## 2.2 Similarity Measures

Let's begin by defining a similarity measure between users. How could we compute the similarity between two users just by using a vector representation of their previous ratings? One popular option is to compute the cosine of the angle between both vectors, i.e., the [cosine similarity](https://en.wikipedia.org/wiki/Cosine_similarity) that we used during the feature engineering last week. Another prevalent option is to compute the [Pearson correlation coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient). But what is the main difference between both?

📝 Inspect the four users below who rated the same three items. We compute the cosine similarity between the users and the Pearson correlation coefficient. Describe similarities and differences between the output of both metrics.

In [ ]:
if __name__ == "__main__":
    user_a = np.array([1, 5, 10])
    user_b = np.array([1, 2, 3])
    user_c = np.array([7, 8, 9])
    user_d = np.array([10, 5, 1])

    example_rating_matrix = np.vstack([
        user_a,
        user_b,
        user_c,
        user_d
    ])

In [ ]:
if __name__ == "__main__":
    print("Cosine similarity between user ratings:")
    print(cosine_similarity(example_rating_matrix).round(2))

In [ ]:
if __name__ == "__main__":
    print("Pearson correlation coefficient between user ratings:")
    print(np.corrcoef(example_rating_matrix).round(2))

<div class="alert alert-info">ADD YOUR ANSWER HERE</div>

## 2.3 User-User Similarity Matrix

Now, let's actually compute the similarity between users in our dataset.

📝 Complete the method below and return a matrix of size (user x user) that contains the pearson correlation between all users in our user-item-rating matrix. Lastly, each user is the most similar to themselves (see the 1.0 score on the diagonal above). Avoid this by setting the similarity of a user to themselves to zero in your matrix.

In [ ]:
def get_user_similarity(user_item_ratings: np.ndarray) -> np.ndarray:
    """
    >>> user_similarity = get_user_similarity(user_item_ratings)
    
    # Check that the resulting matrix is of size user x user
    >>> user_similarity.shape
    (1710, 1710)
    
    # Check that users are not similar to themselves
    >>> (np.diag(user_similarity) == np.zeros((1710))).all()
    True
    
    # Check that similarty matrix does not contain invalid values
    >>> np.isnan(user_similarity).sum() == 0
    True
    
    # Check pearson correlation
    >>> test_user_item_ratings = np.array([[1, 5, 11], [9, 10, 11], [4, 3, 2]])
    >>> expected_user_similarity = np.array([[ 0.,  1., -1.], [ 1.,  0., -1.], [-1., -1.,  0.]])
    >>> actual_user_similarity = get_user_similarity(test_user_item_ratings).round(1)
    >>> np.array_equal(actual_user_similarity, expected_user_similarity)
    True
    """
    user_similarity = np.array([[]])
    
    # ADD YOUR CODE HERE
    
    return user_similarity

In [ ]:
if __name__ == "__main__":
    test(get_user_similarity)
    user_similarity = get_user_similarity(user_item_ratings)

## 2.4 Finding K-Nearest Neighbors

Next, let's use our similarity matrix to find the most related users (neighbors) to a given user we want to recommend a book to.

📝 Find the `k` nearest users with the highest pearson correlation with a given user. Note that you can access the user_similarity matrix that we just created using the `similarity_matrix` parameter in the function below. Return an array with the ids of neighboring users `neighbor_ids` (sorted by decreasing similarity) as well as an array of their `similarities`:

In [ ]:
def get_nearest_neighbors(similarity_matrix: np.ndarray, user_id: int, k: int = 100) -> Tuple[np.ndarray, np.ndarray]:
    """
    # Test that user is not part of the neighbors array
    >>> test_neighbors, test_similarity = get_nearest_neighbors(user_similarity, user_id=20, k=100)
    >>> all(test_neighbors != 20)
    True
    
    >>> test_neighbors, test_similarity = get_nearest_neighbors(user_similarity, user_id=0, k=3)
    >>> test_neighbors
    array([ 430, 1586,  127])
    >>> test_similarity.round(3)
    array([0.229, 0.175, 0.167])
    
    >>> test_neighbors, test_similarity = get_nearest_neighbors(user_similarity, user_id=1500, k=5)
    >>> test_neighbors
    array([ 475,  744,   18, 1057, 1416])
    >>> test_similarity.round(3)
    array([0.152, 0.132, 0.12 , 0.114, 0.11 ])
    """
    neighbor_ids = np.array([])
    similarities = np.array([])
    
    # ADD YOUR CODE HERE
    
    return neighbor_ids, similarities

In [ ]:
if __name__ == "__main__":
    test(get_nearest_neighbors)

## 2.5 Mean Rating Recommendations

We should be all set now to actually make some recommendations. The simplest KNN-based collaborative filtering method is just to compute the average item ratings of our nearest neighbors and return them as a sorted list.

Let $n \in N$ be a neighbor in our set of nearest neighbors $N$ for a given user. For each item in our dataset $i$ compute the average rating of all neighbors. If a neighbor did not rate an item consider their rating as $0$.

$\text{rating(i)} = \frac{1}{|N|} \sum_{n \in N} r_{i,n}$

📝 Complete the method below and compute the top `n_results` book recommendations for a given user. Rank the recommendations by the average rating of the closest k neighbors. As in previous weeks, return a list of tuples containing the book title and the ranked score (rounded to 4 decimal places), break ties by sorting alphabetically by title. Lastly, ensure that your results do not contain books that the user already has rated. Only use the user_item_ratings matrix and do not access the original dataframe to fetch the user's history.

In [ ]:
def get_mean_rating_recommendations(
    user_item_ratings: np.ndarray,
    similarity_matrix: np.ndarray,
    user_id: int,
    k_neighbors: int,
    n_results: int,
    id2title: Dict[int, str]
) -> List[Tuple[str, float]]:
    """
    # Test that recommendations do not contain books from the user's history
    >>> test_recommendations = get_mean_rating_recommendations(user_item_ratings, user_similarity, 1, k_neighbors=1, n_results=3, id2title=id2title)
    >>> user_history = [('Harry Potter and the Chamber of Secrets (Book 2)', 11.0), ('Harry Potter and the Goblet of Fire (Book 4)', 11.0), ('Harry Potter and the Prisoner of Azkaban (Book 3)', 11.0)]
    >>> assert test_recommendations != user_history
    
    >>> get_mean_rating_recommendations(user_item_ratings, user_similarity, 971, k_neighbors=10, n_results=3, id2title=id2title)
    [('Anne of the Island', 2.1), ("Anne's House of Dreams", 2.0), ('Holes (Yearling Newbery)', 2.0)]
    
    >>> get_mean_rating_recommendations(user_item_ratings, user_similarity, 971, k_neighbors=50, n_results=3, id2title=id2title)
    [('The Lovely Bones: A Novel', 0.9), ('Eyes of the Dragon', 0.76), ('The Queen of the Damned (Vampire Chronicles (Paperback))', 0.74)]
    
    >>> get_mean_rating_recommendations(user_item_ratings, user_similarity, 1009, k_neighbors=50, n_results=3, id2title=id2title)
    [('The Da Vinci Code', 1.26), ('Killing Dance (Anita Blake Vampire Hunter (Paperback))', 0.94), ('Cause of Death', 0.78)]
    
    >>> get_mean_rating_recommendations(user_item_ratings, user_similarity, 500, k_neighbors=50, n_results=5, id2title=id2title)
    [('The Lovely Bones: A Novel', 1.68), ('Good in Bed', 1.56), ('A Bend in the Road', 1.1), ('Message in a Bottle', 0.88), ("Suzanne's Diary for Nicholas", 0.8)]
    """
    titles = []
    
    # ADD YOUR CODE HERE
    
    return titles[:n_results]

In [ ]:
if __name__ == "__main__":
    test(get_mean_rating_recommendations)

## 2.6 Similarity-weighted Rating Recommendations

The method above is quite simple. Find a group of similar users and recommend their overall favorite books. However, in the method above, we treat all neighbors the same regardless of their degree of similarity. But intuitively, users that are more similar to me should have a higher impact on my recommendations than the ones with a very different taste. Thus, let's introduce similarity weighted recommendations.

Let $s_n$ be the similarity of a neighbor with our current user. We compute a weighted average by weighting each neighbor's rating with their similarity to our user:

$\text{rating(i)} = \large\frac{\sum_{n \in N} s_{n} \cdot r_{i,n}}{\sum_{n \in N} |s_{n}|}$


Note that we normalize by the sum of all similarities in the denominator. Given that the Pearson correlation can be negative, we use the absolute similarity value in the denominator here.

📝 Implement the similarity weighted user-user recommendation below. As before, return the top n results with title and score (breaking score ties by title ascendingly). Ensure not to return books that the user has already rated before and lastly, round all returned scores to 4 decimal places.

In [ ]:
def get_weighted_rating_recommendations(
    user_item_ratings: np.ndarray,
    similarity_matrix: np.ndarray,
    user_id: int,
    k_neighbors: int,
    n_results: int,
    id2title: Dict[int, str]
) -> List[Tuple[str, float]]:
    """
    # Test that recommendations do not contain books from the user's history
    >>> test_recommendations = get_weighted_rating_recommendations(user_item_ratings, user_similarity, 971, k_neighbors=1, n_results=3, id2title=id2title)
    >>> user_history = [('Harry Potter and the Chamber of Secrets (Book 2)', 11.0), ('Harry Potter and the Goblet of Fire (Book 4)', 11.0), ('Harry Potter and the Prisoner of Azkaban (Book 3)', 11.0)]
    >>> assert test_recommendations != user_history
    
    >>> get_weighted_rating_recommendations(user_item_ratings, user_similarity, 971, k_neighbors=10, n_results=3, id2title=id2title)
    [('Anne of the Island', 1.8494), ('Holes (Yearling Newbery)', 1.8016), ("Anne's House of Dreams", 1.7161)]
    
    >>> get_weighted_rating_recommendations(user_item_ratings, user_similarity, 971, k_neighbors=50, n_results=3, id2title=id2title)
    [('The Lovely Bones: A Novel', 0.901), ('Eyes of the Dragon', 0.8939), ('The Queen of the Damned (Vampire Chronicles (Paperback))', 0.7903)]
    
    >>> get_weighted_rating_recommendations(user_item_ratings, user_similarity, 50, k_neighbors=50, n_results=3, id2title=id2title)
    [('The Firm', 1.1842), ('To Kill a Mockingbird', 0.7558), ('The Lovely Bones: A Novel', 0.7126)]
    
    >>> get_weighted_rating_recommendations(user_item_ratings, user_similarity, 1599, k_neighbors=500, n_results=3, id2title=id2title)
    [('Fahrenheit 451', 1.0806), ('Jurassic Park', 0.7894), ('Fear and Loathing in Las Vegas : A Savage Journey to the Heart of the American Dream', 0.7115)]
    """
    titles = []
    
    # ADD YOUR CODE HERE
    
    return titles[:n_results]

In [ ]:
if __name__ == "__main__":
    test(get_weighted_rating_recommendations)

# Part III Item-Item Collaborative Filtering

In the previous part, we looked at user-user relationships and recommended books that people similar to us also enjoyed. In this part, we look at item-item relationships to recommend books that are similar to what we enjoyed in the past.

### 3.1 Item-Item Similarity Matrix
Let's begin again with our user-item-rating matrix. But this time, compute the similarity between all items so that the returning matrix is of shape (items x items).

📝 Complete the method below and return a matrix of size (item x item) that contains the Pearson correlation between all items in our user-item-rating matrix. Set the similarity of an item to itself to zero.

<div class="alert alert-warning">
💡 Tip: Try out what flipping our user_item_ratings matrix over the diagonal does to the similarity calculation.
</div>

In [ ]:
def get_item_similarity(user_item_ratings: np.ndarray) -> np.ndarray:
    """
    # To make sure that we can test from with CodeGrade, we only take a third of the user_item_ratings
    >>> item_similarity = get_item_similarity(user_item_ratings[:, ::3])
    
    # Check that the resulting matrix is of size user x user
    >>> item_similarity.shape
    (4351, 4351)
    
    # Check that users are not similar to themselves
    >>> (np.diag(item_similarity) == np.zeros((4351))).all()
    True
    
    # Check that similarty matrix does not contain invalid values
    >>> np.isnan(item_similarity).sum() == 0
    True
    
    # Check pearson correlation
    >>> test_user_item_ratings = np.array([[1, 5, 11, 20], [9, 10, 11, 12], [4, 3, 2, 1]])
    >>> expected_item_similarity = np.array([[ 0. ,  0.8,  0.1, -0.3], [ 0.8,  0. ,  0.7,  0.4], [ 0.1,  0.7,  0. ,  0.9], [-0.3,  0.4,  0.9,  0. ]])
    >>> actual_item_similarity = get_item_similarity(test_user_item_ratings).round(1)
    >>> np.array_equal(actual_item_similarity, expected_item_similarity)
    True
    """
    item_similarity = np.array([[]])
    
    # ADD YOUR CODE HERE
    
    return item_similarity

In [ ]:
if __name__ == "__main__":
    test(get_item_similarity)
    item_similarity = get_item_similarity(user_item_ratings[:, ::3])

## 3.2 Item-Item Recommendations

Item-based recommendation is build on the notion that users will like something similar to what they liked in the past. Imagine for example a similarity matrix of three items A, B, and C:

$$
\begin{bmatrix} 
	1 & 0.5 & 0  \\
	0.5 & 1 & 0.3  \\
    0 & 0.3 & 1    \\
\end{bmatrix}
$$

In this artificial setting, items A and B have a similarity of 0.5 and item B and C of 0.3. Items A and C have no relation. If we'd know that a user liked item B and gave a rating of 10, item-based collaborative filtering would then assume through item similarity that our user would rate item A with a score of 5 and item C with 3.

The k nearest neighbors in this model are the number of most similar items for each given item in the rating matrix that would be considered. E.g., in the example above, k=1 would mean that only item A is retrieved with its score of 5, since it is the most similar to item B.

If we set k to all items in our dataset, we essentially compute the dot product between our user's rating vector and the item similarity matrix. For simplicity, let's implement that version of item-item recommendation below and ignore the parameter k.

📝 Complete the method below and implement an item-item collaborative filtering recommendation. Retrieve the user's rating vector and rank items by the result of the dot product of that user vector with the item similarity matrix (no normalization applied for simplicity). Ignore again items that were already in the user's history and break ties in the resulting list by sorting by title. Round all resulting scores to 4 decimal places.

In [ ]:
def get_item_item_recommendations(
    user_item_ratings: np.ndarray,
    similarity_matrix: np.ndarray,
    user_id: int,
    n_results: int,
    id2title: Dict[int, str]
) -> List[Tuple[str, float]]:
    """
    >>> get_item_item_recommendations(user_item_ratings[:, ::3], item_similarity, user_id=971, n_results=3, id2title=id2title)
    [('From the Dust Returned: A Novel', 12.6822), ('Junky', 12.5776), ('The Rival (Rival)', 11.7701)]
    
    >>> get_item_item_recommendations(user_item_ratings[:, ::3], item_similarity, user_id=653, n_results=3, id2title=id2title)
    [('The Indictment', 0.9135), ('The Last Convertible', 0.4855), ('Crown Duel (Smith, Sherwood. Crown and Court Duet, Bk. 1-2.)', 0.36)]
    
    >>> get_item_item_recommendations(user_item_ratings[:, ::3], item_similarity, user_id=151, n_results=3, id2title=id2title)
    [('Northanger Abbey (English Library)', 6.2228), ('A Secret Affair', 5.944), ('Soul Mates', 5.7874)]
    
    >>> get_item_item_recommendations(user_item_ratings[:, ::3], item_similarity, user_id=1141, n_results=5, id2title=id2title)
    [('The Hours : A Novel', 5.956), ('The Bestseller', 5.7026), ("When You Look Like Your Passport Photo, It's Time to Go Home", 4.8398), ("The French Lieutenant's Woman (French Lieutenant's Woman)", 3.9785), ("London's Perfect Scoundrel : Lessons in Love (Lessons in Love)", 3.9451)]
    """
    titles = []
    
    # ADD YOUR CODE HERE
    
    return titles[:n_results]

In [ ]:
if __name__ == "__main__":
    test(get_item_item_recommendations)

## 3.3 Comparing Approaches

To end this assignment, let's think about when we should use different recommendation paradigmns.

❓Your online shop for used Nintendo games is not taking off. So far you have 238 games in stock but last month only 18 users logged in and used your website. What is more memory efficient item-item or user-user recommendation? Does your answer change once you have 500 visitors a month?

<div class="alert alert-info">ADD YOUR ANSWER HERE</div>

❓Popularity bias is the tendency of recommender systems to overly recommend very popular items. Compare collaborative filtering with content-based recommendation, which is more prone to this bias?

<div class="alert alert-info">ADD YOUR ANSWER HERE</div>